In [1]:
import os
import time
import argparse
import numpy as np

import re
from numpy.linalg import norm

def read_docs(file_name):
    print('read documents')
    print('-'*50)
    docs = []
    fp = open(file_name, 'r')
    for line in fp:
        arr = re.split('\s', line[:-1])
        arr = filter(None, arr)
        arr = [int(idx) for idx in arr]
        docs.append(arr)
    fp.close()
    
    return docs

def read_vocab(file_name):
    print('read vocabulary')
    print('-'*50)
    vocab = []
    fp = open(file_name, 'r')
    for line in fp:
        arr = re.split('\s', line[:-1])
        vocab.append(arr[0])
    fp.close()

    return vocab

def calculate_PMI(AA, topKeywordsIndex):
    '''
    Reference:
    Short and Sparse Text Topic Modeling via Self-Aggregation
    '''
    D1 = np.sum(AA)
    n_tp = len(topKeywordsIndex)
    PMI = []
    for index1 in topKeywordsIndex:
        for index2 in topKeywordsIndex:
            if index2 < index1:
                if AA[index1, index2] == 0:
                    PMI.append(0.0)
                else:
                    C1 = np.sum(AA[index1])
                    C2 = np.sum(AA[index2])
                    PMI.append(np.log(AA[index1,index2]*D1/C1/C2))
    avg_PMI = 2.0*np.sum(PMI)/float(n_tp)/(float(n_tp)-1.0)

    return avg_PMI

class SeaNMFL1(object):
    def __init__(
        self,
        A, S, 
        IW1=[], IW2=[], IH=[],
        alpha=1.0, beta=0.1, n_topic=10, max_iter=100, max_err=1e-3, 
        rand_init=True, fix_seed=False):
        '''
        0.5*||A-WH^T||_F^2+0.5*alpha*||S-WW_c^T||_F^2+0.5*beta*||W||_1^2
        W = W1
        Wc = W2
        '''
        if fix_seed: 
            np.random.seed(0)

        self.A = A
        self.S = S

        self.n_row = A.shape[0]
        self.n_col = A.shape[1]

        self.n_topic = n_topic
        self.max_iter = max_iter
        self.alpha = alpha
        self.beta = beta
        self.B = np.ones([self.n_topic,1])
        self.max_err = max_err

        if rand_init:
            self.nmf_init_rand()
        else:
            self.nmf_init(IW1, IW2, IH)
        self.nmf_iter()

    def nmf_init_rand(self):
        self.W1 = np.random.random((self.n_row, self.n_topic))
        self.W2 = np.random.random((self.n_row, self.n_topic))
        self.H = np.random.random((self.n_col, self.n_topic))

        for k in range(self.n_topic):
            self.W1[:, k] /= norm(self.W1[:, k])
            self.W2[:, k] /= norm(self.W2[:, k])

    def nmf_init(self, IW1, IW2, IH):
        self.W1 = IW1
        self.W2 = IW2
        self.H = IH

        for k in range(self.n_topic):
            self.W1[:, k] /= norm(self.W1[:, k])
            self.W2[:, k] /= norm(self.W2[:, k])

    def nmf_iter(self):
        loss_old = 1e20
        print('loop begin')
        start_time = time.time()
        for i in range(self.max_iter):
            self.nmf_solver()
            loss = self.nmf_loss()
            if loss_old-loss < self.max_err:
                break
            loss_old = loss
            end_time = time.time()
            print('Step={}, Loss={}, Time={}s'.format(i, loss, end_time-start_time))

    def nmf_solver(self):
        '''
        using BCD framework
        '''
        epss = 1e-20
        # Update W1
        AH = np.dot(self.A, self.H)
        SW2 = np.dot(self.S, self.W2)
        HtH = np.dot(self.H.T, self.H)
        W2tW2 = np.dot(self.W2.T, self.W2)
        W11 = self.W1.dot(self.B)

        for k in range(self.n_topic):
            num0 = HtH[k,k]*self.W1[:,k] + self.alpha*W2tW2[k,k]*self.W1[:,k]
            num1 = AH[:,k] + self.alpha*SW2[:,k]
            num2 = np.dot(self.W1, HtH[:,k]) + self.alpha*np.dot(self.W1, W2tW2[:,k]) + self.beta*W11[0]
            self.W1[:,k] = num0 + num1 - num2
            self.W1[:,k] = np.maximum(self.W1[:,k], epss) # project > 0
            self.W1[:,k] /= norm(self.W1[:,k]) + epss # normalize
        # Update W2
        W1tW1 = self.W1.T.dot(self.W1)
        StW1 = np.dot(self.S, self.W1)
        for k in range(self.n_topic):
            self.W2[:,k] = self.W2[:,k] + StW1[:,k] - np.dot(self.W2, W1tW1[:,k])
            self.W2[:,k] = np.maximum(self.W2[:,k], epss)
        #Update H
        AtW1 = np.dot(self.A.T, self.W1)
        for k in range(self.n_topic):
            self.H[:,k] = self.H[:,k] + AtW1[:,k] - np.dot(self.H, W1tW1[:,k])
            self.H[:,k] = np.maximum(self.H[:,k], epss) 

    def nmf_loss(self):
        '''
        Calculate loss
        '''
        loss = norm(self.A - np.dot(self.W1, np.transpose(self.H)), 'fro')**2/2.0
        if self.alpha > 0:
            loss += self.alpha*norm(np.dot(self.W1, np.transpose(self.W2))-self.S, 'fro')**2/2.0
        if self.beta > 0:
            loss += self.beta*norm(self.W1, 1)**2/2.0
        
        return loss
    
    def get_lowrank_matrix(self):
        return self.W1, self.W2, self.H
    
    def save_format(self, W1file='W.txt', W2file='Wc.txt', Hfile='H.txt'):
        np.savetxt(W1file, self.W1)
        np.savetxt(W2file, self.W2)
        np.savetxt(Hfile, self.H)
        
'''
Topic Modeling via NMF
'''
class NMF(object):
    def __init__(
        self, 
        A, IW=[], IH=[],
        n_topic=10, max_iter=100, max_err=1e-3,
        rand_init=True):
        '''
        A = WH^T
        '''
        self.A = A
        self.n_row = A.shape[0]
        self.n_col = A.shape[1]
        
        self.n_topic = n_topic
        self.max_iter = max_iter
        self.max_err = max_err

        self.obj = []
        if rand_init:
            self.nmf_init_rand()
        else:
            self.nmf_init(IW, IH)
        self.nmf_iter()
    
    def nmf_init_rand(self):
        self.W = np.random.random((self.n_row, self.n_topic))
        self.H = np.random.random((self.n_col, self.n_topic))

        for k in range(self.n_topic):
            self.W[:,k] /= norm(self.W[:,k])
                
    def nmf_init(self, IW, IH):
        self.W = IW
        self.H = IH

        for k in range(self.n_topic):
            self.W[:,k] /= norm(self.W[:,k])

    def nmf_iter(self):
        loss_old = 1e20
        print('loop begin')
        start_time = time.time()
        for i in range(self.max_iter):
            self.nmf_solver()
            loss = self.nmf_loss()
            self.obj.append(loss)

            if loss_old-loss < self.max_err:
                break
            loss_old = loss
            end_time = time.time()
            print('Step={}, Loss={}, Time={}s'.format(i, loss, end_time-start_time))
        print('loop end')
                
    def nmf_solver(self):
        '''
        regular NMF without constraint.
        Block Coordinate Decent
        '''
        epss = 1e-20
        
        HtH = self.H.T.dot(self.H)
        AH = self.A.dot(self.H)
        for k in range(self.n_topic):
            tmpW = self.W[:,k]*HtH[k,k] + AH[:,k] - np.dot(self.W, HtH[:,k])
            self.W[:,k] = np.maximum(tmpW, epss)
            self.W[:,k] /= norm(self.W[:,k]) + epss
        
        WtW = self.W.T.dot(self.W)
        AtW = self.A.T.dot(self.W)
        for k in range(self.n_topic):
            self.H[:,k] = self.H[:,k]*WtW[k,k] + AtW[:,k] - np.dot(self.H, WtW[:,k])
            self.H[:,k] = np.maximum(self.H[:,k], epss)

    def nmf_loss(self):
        loss = norm(self.A - np.dot(self.W, np.transpose(self.H)), 'fro')**2/2.0
        return loss
    
    def get_loss(self):
        return np.array(self.obj)
    
    def get_lowrank_matrix(self):
        return self.W, self.H
    
    def save_format(self, Wfile='W.txt', Hfile='H.txt'):
        np.savetxt(Wfile, self.W)
        np.savetxt(Hfile, self.H)



In [2]:
corpus_file="wedoc_term_mat.txt"
vocab_file="word_embedding_vocab.txt"
model="seanmf"
max_iter=500
n_topics=4
alpha=0.1
beta=0.0
max_err=0.1
fix_seed=True

In [3]:
docs = read_docs(corpus_file)
vocab = read_vocab(vocab_file)
n_docs = len(docs)
n_terms = len(vocab)
print('n_docs={}, n_terms={}'.format(n_docs, n_terms))

read documents
--------------------------------------------------
read vocabulary
--------------------------------------------------
n_docs=28220, n_terms=2000


In [4]:
tmp_folder = 'seanmf_results/'
if not os.access(tmp_folder, os.F_OK):
    os.mkdir(tmp_folder)

In [6]:
if model.lower() == 'seanmf':
    print('calculate co-occu·rance matrix')
    dt_mat = np.zeros([n_terms, n_terms])
    for itm in docs:
        for kk in itm:
            for jj in itm:
                dt_mat[int(kk)-1,int(jj)-1] += 1.0
    print('co-occur done')
    print('-'*50)
    print('calculate PPMI')
    D1 = np.sum(dt_mat)
    SS = D1*dt_mat
    for k in range(n_terms):
        SS[k] /= np.sum(dt_mat[k])
    for k in range(n_terms):
        SS[:,k] /= np.sum(dt_mat[:,k])
    dt_mat = [] # release memory
    SS[SS==0] = 1.0
    SS = np.log(SS)
    SS[SS<0.0] = 0.0
    print('PPMI done')
    print('-'*50)
    
    print('read term doc matrix')
    dt_mat = np.zeros([n_terms, n_docs])
    for k in range(n_docs):
        for j in docs[k]:
            dt_mat[j-1, k] += 1.0
    print('term doc matrix done')
    print('-'*50)
    
    model = SeaNMFL1(
        dt_mat, SS,  
        alpha=alpha, 
        beta=beta, 
        n_topic=n_topics, 
        max_iter=max_iter, 
        max_err=max_err,
        fix_seed=fix_seed)

    model.save_format(
        W1file=tmp_folder+'4W.txt',
        W2file=tmp_folder+'4Wc.txt',
        Hfile=tmp_folder+'4H.txt')


calculate co-occu·rance matrix
co-occur done
--------------------------------------------------
calculate PPMI
PPMI done
--------------------------------------------------
read term doc matrix
term doc matrix done
--------------------------------------------------
loop begin
Step=0, Loss=177777.22631996515, Time=1.4610865116119385s
Step=1, Loss=174526.07387571628, Time=2.4356794357299805s
Step=2, Loss=171783.09570255777, Time=3.345795154571533s
Step=3, Loss=168854.26999361458, Time=4.142989635467529s
Step=4, Loss=167448.956526487, Time=4.94830322265625s
Step=5, Loss=166685.37848895398, Time=5.7555625438690186s
Step=6, Loss=166115.99172322766, Time=6.569148778915405s
Step=7, Loss=165713.55944281735, Time=7.496948719024658s
Step=8, Loss=165496.57248496404, Time=8.478071689605713s
Step=9, Loss=165381.2102616236, Time=9.282793998718262s
Step=10, Loss=165315.06974200124, Time=10.062366724014282s
Step=11, Loss=165273.73345314, Time=10.82845950126648s
Step=12, Loss=165247.5768593724, Time=11.